In [ ]:
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

def extract_item_links(html_path):
    # HTML 파일 읽기
    with open(html_path, 'r', encoding='utf-8') as f:
        html = f.read()

    # &nbsp; 제거 및 공백 변환
    html = re.sub(r'(&nbsp;)+', ' ', html)

    # "ITEM X." 패턴 찾기 (예: ITEM 1., ITEM 2., ITEM 5.)
    found_items = re.findall(r'item\s+?\d\.', html, flags=re.IGNORECASE)
    print("추출된 ITEM 목록:", found_items)

    # 특정 인덱스의 ITEM 정보 가져오기
    selected_indices = [0, 1, 4, 7]  # 1번째, 2번째, 5번째, 8번째 (0-based index)
    selected_items = [found_items[i] for i in selected_indices if i < len(found_items)]
    print("선택된 ITEM 목록:", selected_items)

    # HTML 파싱
    soup = BeautifulSoup(html, 'html.parser')

    # 선택된 ITEM과 동일한 이름을 가진 <a> 태그 찾기
    item_links = []
    for item in selected_items:
        matching_link = soup.find('a', string=re.compile(re.escape(item), re.IGNORECASE))
        if matching_link:
            href_value = matching_link.get('href')
            if href_value and href_value.startswith('#'):
                href_value = href_value[1:]  # '#' 제거
            item_links.append(href_value)
        else:
            item_links.append(None)

    # ranges 리스트 구성
    ranges = [(item_links[0], item_links[1]), (item_links[2], item_links[3])]
    
    return soup, ranges

def extract_table_from_html(table, table_index):
    """ HTML 테이블을 판다스 DataFrame으로 변환하고 Markdown 형식으로 반환 """
    try:
        df = pd.read_html(str(table))[0]
        print(df)
        return f"### {table_index}\n" + df.to_markdown(index=False) + "\n"
    except ValueError:
        return "테이블을 찾을 수 없습니다."

def extract_text_and_tables_from_multiple_ranges(soup, ranges):
    """ 여러 개의 특정 div 또는 a 범위 내에서 span 태그의 텍스트와 table 태그의 데이터를 수집 """
    combined_texts = []
    table_count = 1
    
    for start_id, end_id in ranges:
        start_element = soup.find(lambda tag: (tag.name == "div" or tag.name == "a") and tag.get("id") == start_id)
        end_element = soup.find(lambda tag: (tag.name == "div" or tag.name == "a") and tag.get("id") == end_id)

        if not start_element or not end_element:
            print(f"지정한 ID의 div 또는 a 태그를 찾을 수 없습니다: {start_id} - {end_id}")
            continue

        all_elements = soup.find_all(lambda tag: tag.name == "div" or tag.name == "a")
        start_index = all_elements.index(start_element)
        end_index = all_elements.index(end_element)
        div_count = end_index - start_index + 1
        print(f"'{start_id}'부터 '{end_id}'까지의 div 또는 a 개수: {div_count}")

        for element in all_elements[start_index:end_index+1]:
            table = element.find("table")
            if table:
                table_text = extract_table_from_html(table, table_count)
                combined_texts.append(f"[TABLE {table_count}]")
                combined_texts.append(table_text)
                table_count += 1
                continue

            # 테이블이 포함된 경우, 그 내부의 <span> 태그는 수집하지 않음
            if element.find("table"):
                continue

            spans = [span for span in element.find_all("span") if not span.find_parent("table")]
            for span in spans:
                text_content = span.get_text(strip=True)
                combined_texts.extend(text_content.split(","))
    
    return combined_texts

def save_results_to_csv(combined_texts, output_path):
    """ 수집된 문장 리스트와 테이블 목록을 하나의 CSV 파일로 저장 """
    df = pd.DataFrame({"Content": combined_texts})
    df.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"결과가 CSV 파일로 저장되었습니다: {output_path}")

# 실행 예시
html_path = r'C:\Users\RMARKET\Desktop\yong\data_folder\crawing_data\result\0000002488\0000002488_10-K_Report_2.html'
soup, ranges = extract_item_links(html_path)
combined_texts = extract_text_and_tables_from_multiple_ranges(soup, ranges)
output_csv_path = r'C:\Users\RMARKET\Desktop\yong\data_folder\crawing_data\parsed_data\0000004127\output2.csv'
save_results_to_csv(combined_texts, output_csv_path)
